<font size=4 color="red">
    WARNING: This is for **Multi Layer Perceptron** (MLP) which is NOT the same as question 3 in week 3 worksheet because I MADE A MISTAKE. <br>
    The worksheet asked for **PERCEPTRON** with sigmoid activation fuction and quartic (x^4) loss. <br> 
    For some reason I thought Perceptron is the same as MLP with single hidden layer, but **IT IS NOT**. 
    <br><br>
    I messed up the terminology. Perceptron is more like fully connected feedforward network (a.k.a. MLP) without any hidden-layer at all, feeding input straight into an activation function.
    <br><br>
    This notebook (and during class) I was working with MLP with 1-hidden layer. I only realised my mistake later.
    I sincerely apologise for the confusion. 
    <br><br>
    See written solution for the answer for Perceptron (it's much much simpler...). 
    <br> 
    Hopefully this note might help with 
    <ul> 
        <li> understanding backprop, </li>
        <li> the trick for absorbing bias into weights </li> 
        <li> Counting number of network parameters </li>
        <li> the gory index-tracking and notation used to simplify it </li>
        <li> implementing MLP in Python yourself </li>
    </ul>
</font>

---
# Gradient Descent for MLP with sigmoid activation and quartic loss
We wish to implement explicitly ourselves a fully connected feedforward network (a.k.a. Multi Layer Perceptron) with 
 * $d$-dimensional inpuut, 
 * $n$-dimensional output and 
 * $L$ number of hidden layers each with $m_1, \dots, m_L$ number of hidden units. 
 * each neuron has activation function $\sigma(x) = 1/ (1 - exp(x))$
 
TODO: picture. 

---
# Messy details of the netwok itself
The neural network defines a **function** 
 * taking $d$-dimensional input, i.e. each input $x = (x_1, \dots, x_d) = (x_i)_{i = 1, \dots, d}$ is a $d$-dimensional vector. 
 * and output $n$ dimensional vector
Meaning, out neural network here would be a function of the following form.
$$
f: \mathbb{R}^d \to \mathbb{R}^n\\
f(\mathbf{x}) = f(x_1, \dots, x_d) = \left(f_1(\mathbf{x}), \dots, f_n(\mathbf{x})\right)
$$

**Input layer to first hidden layer** 

Since we have $d$-dimensional input, the input layer has $d$ nodes. So, each node in the first hidden layer is taking all $d$ values coming from the input layer. Let $H^1_{j}$ denote the $j^{th}$ neuron of the first input layer. Then, 
$$ H^1_i(x_1, \dots, x_d) = \sigma\left(\sum_{j = 1}^d W^{(1)}_{ij}x_i + b^{(1)}_{i}\right)$$
Note:
 * Note that the activation function only cares about a **scalar** input!! It takes in one number and output one number. Verify that the input to $\sigma$ above is actually a scalar.  
 * the superscript in $W^{(l)}, b^{(l)}$ is to remind ourselves that we coming from layer $0$ (the input) into layer $1$. 
 * I am using a fixed indexing variable naming here to keep track of what objects are being indexed: 
   * $l$ is the index for hidden layer $l = 0$ (i.e. the input layer) above and $l = 0, \dots L$. 
   * $i$ is the index for the neurons in hidden layer $l + 1$, so $i = 1, \dots, m_{l + 1}$. 
   * $j$ is the index for the neurons in hidden layer $l$, so $j = 1, \dots, m_l$. We need this because the input to layer $l + 1$ depends (and only depends) on the neurons in the layer before it and the connecting weights $W^{(l)}_{ij}$. 
 * We use capital letter $W$ because it is going to look a whole lot like a marix later on. 
 * The reason the index $ij$ is labelled in that order is also because they correspond to the row and column index of the matrix later on. 


**Hidden Layer in between**   
The equation for the hidden node at layer $l > 0$ generalise in a straightforward manner from the above:
$$
H^{l + 1}_i = \sigma\left(\sum_{j = 1}^{m_l} W^{(l)}_{ij}H^{l}_i + b^{(l)}_{i}\right)
$$
we only need to change the input $x$ into input from a previous layer $h^{l}$ and be careful of the bounds of the index $i = 1, \dots, m_{l + 1}$ and $j = 1, \dots, m_l$. 


**Hidden layer to output layer**  

Since the output dimension is $n$, the output layers have $n$ nodes, each taking input from the $m_L$ outputs from the last hidden layer, the expression for each of the output nodes $f_i$, $i = 1, \dots, n$ is given by
$$
f_i = \sigma\left(\sum_{j = 1}^M W^{(L)}_{ij}H^{L}_j + b^{(L)}_{i} \right)
$$




**The full unnecessarily messy functional form**  
With the definition above, this MLP network is just a function $f = (f_1, \dots, f_n)$, with the following complicated looking expression 
$$
f_i(x_1, \dots, x_d) 
= \sigma\left(
    \sum_{j_L = 1}^{m_L} W^{(L)}_{ij_L}
        \sigma\left(
            \sum_{j_{L-1} = 1}^{m_{L-1}} W^{(L-1)}_{j_{L-1}i}
                \sigma\left( \dots
                    \sigma\left(\sum_{j_0 = 1}^{d} W^{(0)}_{j_1j_0}x_{j_0} + b^{(0)}_{j_1}
                    \right) + b^{(1)}_{j_2}
                \right) + \dots
            \right) + b^{(L-1)}_{j_{L}} 
\right) + b^{L}_i \\
$$

<font color="red">
You can see why we choose to draw networks rather than writing out the functional form. The network makes it clear what nodes depends on what weights. 
</font>


## Simplification
Just to simplify the expression for neural network we introduce the following convention  

**Elementwise notation**  
When you use `numpy.log([1, 2, 3])`, numpy internally does `[log(1), log(2), log(3)]` for you, meaning the function is applyed **element-by-element**. The reason is that instead of doing `for elem in array: log(elem)` in python, the loop actually occur in $C$ which is a lot faster. This notation is common and worth knowing: when you have function $\sigma(x)$ that is suppose to take one input and give one output, but the written input is a vector (or array of any dimension) it is understood that we mean apply the function **element-by-element** and output an array of the same dimension. 

**Matrix multiplication**  
Notice that the sum $\sum_{i = 1}^d W_{ji}x_i$ is the same as the dot-product of the $j^{th}$ of the matrix $W$ with the vector $\mathbf{x} = (x_1, \dots, x_d)$. So, in vector form, we have 
$$
\text{Vector output $y_j$ for $j = 1 \dots k$} = \left(\begin{matrix}
y_1 \\
\vdots \\
y_k
\end{matrix}\right) = W\mathbf{x}
$$


**Absorbing the bias term in to the weights**  
The bias term is a little confusing for me. For example is the bias applied before or after the activation? Both should work in practice but I think the consensus is that it is applied before the activation (the expression above is with this assumption). With that in mind, we can do a trick that simplifies the presentation even more. 

We will use the same trick people use in [linear regression](https://en.wikipedia.org/wiki/Linear_regression#Formulation). We add a $0^{th}$ dimension to each layer (including input layer), which is a constant vector of ones. This amounts to adding another node to the input layer so that it becomes $(1, x_1, \dots, x_d)$ instead of just $(x_1, \dots, x_d)$. With this we can rewrite the 

$$
\left(
\begin{matrix}
\sum_{i = 1}^d W_{1i}x_i + b_{1} \\
\dots \\
\sum_{i = 1}^d W_{ki}x_i + b_{k}
\end{matrix}
\right) = 
\left(
\begin{matrix}
W_{11} & W_{12} & \dots & W_{1d} \\
W_{21} & W_{22} & \dots & W_{2d} \\
\ddots & & \\
\dots & W_{ji} & \dots \\
W_{k1} & & \dots & W_{kd}
\end{matrix}
\right)
\left(\begin{matrix} x_1 \\ \vdots \\ x_d \end{matrix}\right) + \left(\begin{matrix} b_1 \\ \vdots \\ b_d \end{matrix}\right) = 
\left(
\begin{matrix}
b_1 & W_{11} & W_{12} & \dots & W_{1d} \\
b_2 & W_{21} & W_{22} & \dots & W_{2d} \\
\ddots & & \\
\dots & W_{ji} & \dots \\
b_k & W_{k1} & & \dots & W_{kd}
\end{matrix}
\right)
\left(\begin{matrix}1 \\ x_1 \\ \vdots \\ x_d \end{matrix}\right) = \overline{W}\overline{x}
$$
where we absorbed the bias vector into the weight matrix $W$ and $\overline{x}$ operation simply add initial element of 1 into a vector. We index those initial addition with the index $0$.


**Shorthand for MLP: Pulling together simplifications**  

So, with the addition of the "bias node" (i.e. a node that just output 1) in the input layer and the hidden-layer, and using elementwise notation for the activation function, we can rewrite 
$$
f(x) = \sigma(W^{(L)}\overline{\sigma(W^{(L-1)}\overline{\sigma(\dots (\overline{\sigma(W^{(0)}\overline{x})}))})})
$$
where the bias vectors $b^{(l)}$ have been absorbed into the weight matrices $W^{(l)}$. 


We shall write our network as $f_W(x)$ where $W$ denote the full $W^{(l)}$ with the bias vectors absorbed. This is to make the explicit that an network prediction on an input $x$ is dependent on the set of **network parameters** $W$.

----
# Loss function
---
Instead of squared-error, we are prompted to use quartic $x^4$ error. 

Let's expand definitions to see what that means: 
 1. Given an input $x$. Remember that we have $d$ dimensional input so it is really $x = (x_1, \dots, x_d)$. 
 2. with the true output $y(x)$, Remember that we have $n$ dimensional output so it is really $y = (y_1, \dots, y_n)$.
 3. We can compute the network prediction by just passing $x$ through our network $f_W$, giving $f_W(x)$. Again, don't forget that our output is $n$-dimensional, so it is really $f_W(x) = (f_W(x)_1, \dots, f_W(x)_n)$. 
 4. We want out error to be proportional to the difference between $y$ and $f(x)$ to the 4th power. We simply sum the difference component-wise and raise it to the 4th power:
 $$
 L_W(x) = \frac{1}{4} \sum_{k = 1}^n (y_k(x) - f_W(x)_k)^4
 $$

Note: 
 * I am using the same indexing variable $k$ for the output dimension which is the same as the size (number of nodes) in the final output layer. 
 * the $1/4$ is added purely to remove the constant later when we differentiate $x^4$. Scaling the loss by a constant does not change where the minimum occurs. 
 * This is the loss for a single input $x$. So if we are doing online learning, this is enough. 
 * If we are doing batch or minibatch gradient descent, we simply take the batch average: 
 $$ L_W(\{X^1, X^2, \dots, X^b, \dots, X^B\}) = \frac{1}{B}\sum_{b = 1}^B L_W(X_b)$$
 where we use $b$ as the batch sample index with $B$ as the batch size. 

----
# Gradient of the Loss Function
----
Since we aim to use SGD to find the minima of the loss function, we need to compute its gradient **with respect to the network parameters**
$$
\nabla_W L_W(x) 
$$
which tells us the direction of greatest *ascent* when we are at a specific parameter $W$ with $-\nabla_W L_W(x)$ being the direction (in parameter space) where we want to step to get the most decrease in loss. This is really just an exercise in using chain-rule and carefull book keeping of various indices. 

Note: 
 * During calculation of the gradient, the input $x$ or the batch of inputs $\{X^1, \dots, X^B\}$ is **FIXED** and 
 * we only care about minimising loss with respect to the **network parameters**, i.e. the thing that is changing during training time. We are searching for the best(-ish) parameters **GIVEN** that we know the output for the batch of inputs. 

---
## Ok, here goes the Gory details...
First, lets note that, since taking derivative commutes with taking (finite) average, 
$$ \frac{\partial}{\partial w} L_W(\{X^1, X^2, \dots, X^b, \dots, X^B\}) = \frac{1}{B}\sum_{b = 1}^B \frac{\partial}{\partial w} L_W(X_b).$$
So we really only need to compute the gradient for the per-input loss $L_W(x)$. 

Lets use the notation 
 * $\partial^{l}_{ij} = \frac{\partial}{\partial W^{l}_{ij}}$ which is the partial derivative with respect to network weight $W^{(l)}_{ij}$ 
 * recall that $W^{(l)}_{ij}$ is the network weight connecting layer node $j$ from layer $l$ to node $i$ of layer $l + 1$
 * remember that $i = 0, 1 \dots m_{l + 1}$ and $j = 0, 1 \dots m_l$, where we absorbed bias terms into the $0^{th}$ index. 



So here goes the chain rule 😭 and lets to the weights for the final layer first... 
$$
\begin{align*}
\partial^{L}_{ij}L_W(x) 
& = \frac{1}{4} \sum_{k = 1}^n \partial^{L}_{ij}(y_{k}(x) - f_W(x)_{k})^4 \\
& = \sum_{k = 1}^n (y_{k}(x) - f_W(x)_{k})^3 \partial^{L}_{ij}f_W(x)_{k} \\
\end{align*}
$$
Fortunately, the variable $W^L_{ij}$ only appears in terms where $k = i$, and so all other terms with $k \neq i$ have zero $\partial^L_{ij}$-derivative, and we have 
$$
\partial^{L}_{ij} L_W(x) = (y_i(x) - f_W(x)_i)^3 \partial^{L}_{ij}f_W(x)_{i} \\
$$
As expected, we need to know the gradient with respect to $i^{th}$ output of the network itself $\partial^{L}_{ij}f_W(x)_i$. 


And that's another chain rule 😭... 
$$
\begin{align*}
& \partial^{L}_{ij}f_W(x)_i \\
&= \partial^{L}_{ij} \sigma\left(\sum_{j' = 0}^{m_L} W^{(L)}_{ij'}H^{L}_{j'} \right) \tag*{notice index starts from 0}\\
&= \sigma'\left(\sum_{j' = 0}^{m_L} W^{(L)}_{ij'}H^{L}_{j'}\right)\, 
   \partial^{L}_{ij}\sum_{j = 0}^{m_L} W^{(L)}_{ij'}H^{L}_{j'} \tag*{chain rule} \\
&= \sigma'\left(\sum_{j' = 0}^{m_L} W^{(L)}_{ij'}H^{L}_{j'}\right)\, 
   \partial^{L}_{ij} W^{(L)}_{ij}H^{L}_{j} \tag*{non-$ij$ terms vanishes} \\
&= \sigma'\left(\sum_{j' = 0}^M W^{(2)}_{ij'}H_{j'}\right)\, H^L_{j}  \tag*{$\partial^{L}_{ij}W^L_{ij} =1$} \\
\end{align*}
$$

So far we have not restrict ourselve to a particular activation function. As long as we know the derivative $\sigma'$ we are good. For sigmoid activation, we have (exercise) $\sigma'(x) = \sigma(x)(1 - \sigma(x))$. So collecting all these together we have 
$$
\begin{align*}
\partial^L_{ij} L_W(x) 
&= (y_i(x) - f_W(x)_i)^3 \partial^L_{ij}f_W(x)_{i} \\
&= (y_i(x) - f_W(x)_i)^3 \sigma\left(\sum_{j' = 0}^M W^{(2)}_{ij'}H_{j'}\right)\left(1 - \sigma\left(\sum_{j' = 0}^M W^{(2)}_{ij'}H_{j'}\right) \right)\, H^L_{j} \\
\end{align*}
$$



## Derivatives of lower layers weights and how backpropagation helps
TODO....

---
# Python Implementation
---
TODO ... 

In [95]:
import numpy as np

def sigmoid(x):
    return 1 / (1 - np.exp(-x))

def sigmoid_diff(x):
    return sigmoid(x) * (1 - sigmoid(x))

def ext(x):
    # extend the vector x by prepending a 1.
    if len(x.shape) == 2: 
        n = x.shape[0]
        return np.concatenate([np.ones((n, 1)), x], axis=1)
    elif len(x.shape) == 1:
        return np.concatenate([[1], x])

class MultiLayerPerceptron:
    
    def __init__(
        self, 
        hidden_layer_sizes,
        input_dim=1,
        output_dim=1,
        activation=sigmoid,
        activation_diff=sigmoid_diff, 
        learning_rate=0.001,
        max_epoch=500, 
        batch_size=100
    ):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.input_dim = input_dim
        self.output_dim = output_dim        
        self.activation = activation
        self.activation_diff = activation_diff
        self.learning_rate = learning_rate
        self.max_epoch = max_epoch
        self.batch_size = batch_size
        
        self.weights = [np.random.rand(input_dim + 1, hidden_layer_sizes[0])]
        last_layer_size = hidden_layer_sizes[0]
        for size in hidden_layer_sizes[1:]:
            shape = (last_layer_size + 1, size) # +1 to add bias node
            self.weights.append(np.random.rand(*shape))
            last_layer_size = size
        self.weights.append(np.random.rand(last_layer_size + 1, output_dim))
        
        self.hidden_layer_values = []
    
    def forward_pass(self, x):
        output = x
        for weight in self.weights:
            output = sigmoid(ext(output) @ weight) # we use row vector multiplication on the left instead.
        return output
    
    def loss(self, x, y):
        pred = self.forward_pass(x)
        return np.sum((y - pred)**4)/4
    
    def backprop(self, x):
        raise NotImplementedError
    
    def grad(self, x, y):
        raise NotImplementedError
    
    def batch_grad(self, X, Y):
        return np.mean(self.grad(x, y) for x, y in zip(X, Y))
    
    def step(self):
        raise NotImplementedError
        
    def fit(self, X, Y):
        raise NotImplementedError
        
    def predict(self, X):
        return self.forward_pass(X)
    

In [96]:
x = np.array([[3, 2, 5], [1, 1, 1]])
y = np.array([1, 2])
clf = MultiLayerPerceptron([10, 5, 5, 5, 2], input_dim=x.shape[1])

clf.forward_pass(x), clf.loss(x, y)

(array([[1.31658437],
        [1.31658346]]),
 0.1140939391636162)